In [15]:
from emtk.fixation_classification import idt_classifier

def read_SMIRed250(root_dir, filename, experiment_id,
                   minimum_duration=50, sample_duration=4, maximum_dispersion=25) -> list:
    """Read tsv file from SMI Red 250 eye tracker

    Parameters
    ----------
    root_dir : str
        Path to directory that contains the asc file.

    filename : str
        Name of asc file.

    experiment_id : str
        Id of the experiment contained in the file.

    Returns
    -------
    list
        List of eye events. Each eye event is represented as a list of eye event features.
    """

    # Reads raw data and sets up
    tsv_file = open(os.path.join(root_dir, filename))
    print("parsing file:", filename.split("/")[-1])
    text = tsv_file.read()
    text_lines = text.split('\n')

    trial_id = 0
    stimuli_name = ""
    raw_fixations = []
    active = False  # Indicates whether samples are being recorded in trials
    # The goal is to skip metadata in the file

    eye_events = []
    samples = []

    # Parses the data into dataframes
    for line in text_lines:
        token = line.split("\t")

        if len(token) < 3:
            continue

        if active:
            # Filter MSG samples if any exist, or R eye is inValid
            if token[1] == "SMP" and token[27] != "-1":
                # Get x and y for each sample (right eye only)
                # [23] R POR X [px]	 '0.00',
                # [24] R POR Y [px]	 '0.00',

                new_sample = samples_list(
                    eye_tracker=EYE_TRACKER,
                    experiment_id=experiment_id,
                    participant_id=experiment_id,
                    filename=filename,
                    trial_id=str(trial_id),
                    stimuli_module=STIMULI_MODULE,
                    stimuli_name=stimuli_name,
                    token=token
                )

                samples.append(new_sample)

                raw_fixations.append(
                    [int(token[0]), float(token[23]), float(token[24])])

        if token[1] == "MSG" and token[3].find(".jpg") != -1:

            if active:
                filter_eye_events = idt_classifier(raw_fixations=raw_fixations,
                                                   minimum_duration=minimum_duration,
                                                   sample_duration=sample_duration,
                                                   maximum_dispersion=maximum_dispersion)
                # TODO saccades

                for timestamp, duration, x_cord, y_cord in filter_eye_events:

                    new_eye_event = eye_event_list(eye_tracker=EYE_TRACKER,
                                                   experiment_id=experiment_id,
                                                   participant_id=experiment_id,
                                                   filename=filename,
                                                   trial_id=str(trial_id),
                                                   stimuli_module=STIMULI_MODULE,
                                                   stimuli_name=stimuli_name,
                                                   duration=duration,
                                                   timestamp=timestamp,
                                                   x0=x_cord,
                                                   y0=y_cord,
                                                   token=token,
                                                   pupil=0,
                                                   eye_event_type="fixation")

                    eye_events.append(new_eye_event)

                trial_id += 1

            # Message: vehicle_java2.jpg
            stimuli_name = token[3].split(' ')[-1]
            raw_fixations = []
            active = True

    # Adds the last trial
    filter_fixations = idt_classifier(raw_fixations=raw_fixations,
                                      minimum_duration=minimum_duration,
                                      sample_duration=sample_duration,
                                      maximum_dispersion=maximum_dispersion)

    for timestamp, duration, x_cord, y_cord in filter_fixations:

        new_eye_event = eye_event_list(eye_tracker=EYE_TRACKER,
                                       experiment_id=experiment_id,
                                       participant_id=experiment_id,
                                       filename=filename,
                                       trial_id=str(trial_id),
                                       stimuli_module=STIMULI_MODULE,
                                       stimuli_name=stimuli_name,
                                       duration=duration,
                                       timestamp=timestamp,
                                       x0=x_cord,
                                       y0=y_cord,
                                       token=token,
                                       pupil=0,
                                       eye_event_type="fixation")

        eye_events.append(new_eye_event)

    return eye_events, samples


In [55]:
import math
import statistics


def idt_classifier(raw_fixations, minimum_duration=50, sample_duration=4, maximum_dispersion=25):
    """I-DT classifier based on page 296 of eye tracker manual:
        https://psychologie.unibas.ch/fileadmin/user_upload/psychologie/Forschung/N-Lab/SMI_iView_X_Manual.pdf

        Notes:
            remember that some data is MSG for mouse clicks.
            some records are invalid with value -1.
            read right eye data only.

    Parameters
    ----------
    raw_fixations : list
        a list of fixations information containing timestamp, x_cord, and y_cord

    minimum_duration : int, optional
        minimum duration for a fixation in milliseconds, less than minimum is considered noise.
        set to 50 milliseconds by default

    sample_duration : int, optional
        Sample duration in milliseconds, this is 4 milliseconds based on this eye tracker

    maximum_dispersion : int, optional
        maximum distance from a group of samples to be considered a single fixation.
        Set to 25 pixels by default

    Returns
    -------
    list
        a list where each element is a list of timestamp, duration, x_cord, and y_cord
    """

    # Create moving window based on minimum_duration
    window_size = int(math.ceil(minimum_duration / sample_duration))

    window_x = []
    window_y = []

    filter_fixation = []

    # Go over all SMPs in trial data
    for timestamp, x_cord, y_cord in raw_fixations:
        #print("data", timestamp, x_cord, y_cord)
        # Filter (skip) coordinates outside of the screen 1920×1080 px
        if x_cord < 0 or y_cord < 0 or x_cord > 1920 or y_cord > 1080:
            continue

        # Add sample if it appears to be valid
        window_x.append(x_cord)
        window_y.append(y_cord)

        # Calculate dispersion = [max(x) - min(x)] + [max(y) - min(y)]
        dispersion = (max(window_x) - min(window_x)) + \
            (max(window_y) - min(window_y))
        #print("dispersion", dispersion)

        # If dispersion is above maximum_dispersion
        if dispersion > maximum_dispersion:
            # Then the window does not represent a fixation
            # Pop last item in window
            window_x.pop()
            window_y.pop()
            #print("dispersion > maximum_dispersion", dispersion, len(window_x), len(window_y))
            # Add fixation to fixations if window is not empty (size >= window_size)
            if len(window_x) == len(window_y) and len(window_x) > window_size:
                # The fixation is registered at the centroid of the window points
                filter_fixation.append(
                    [timestamp, len(window_x) * sample_duration, statistics.mean(window_x), statistics.mean(window_y)])

            window_x = []
            window_y = []

    return filter_fixation


In [64]:
from emtk import parsers
import os
import csv
import numpy as np

minimum_duration = 50
sample_duration = 1000 / 120  # Tobii X3-120 has 120 Hz
maximum_dispersion = 25

folder_path = 'X:\\Uni\\Bachelorarbeit\\RQ3 Materials\\good\\22\\Sessions'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('.tsv')]

# Create a CSV file to store the results
output_csv = 'bachelorthesis/emip_fixations.csv'

list_of_eye_events = dict()

# Iterate over the files and apply your function to each one
for file_name in files:
    experiment_id = 1

    file_path = os.path.join(folder_path, file_name)

    data_list = []
    with open(file_path, 'r', newline='') as tsvfile:
        tsvreader = csv.reader(tsvfile, delimiter='\t')
        next(tsvreader)  # Skip the header row
        for row in tsvreader:
            timestamp = row[0]  # Assuming timestamp is in the first column
            x_cord = row[6]  # Assuming X coordinate is in the third column
            y_cord = row[7]  # Assuming Y coordinate is in the fourth column

            if timestamp and x_cord and y_cord:
                data_list.append([int(timestamp), float(x_cord), float(y_cord)])

    filter_eye_events = idt_classifier(
        raw_fixations=data_list,
        minimum_duration=minimum_duration,
        sample_duration=sample_duration,
        maximum_dispersion=maximum_dispersion
    )

    list_of_eye_events[file_name] = filter_eye_events

# Open the CSV file for writing
with open(output_csv, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header row to the CSV file
    csv_writer.writerow(['File Name', 'Saccades', 'Fixations', 'Total Fixation Duration', 'Average Fixation Duration'])

    # Iterate over the files and apply your function to each one
    for file_name, eye_events in list_of_eye_events.items():
        fixation_durations = [fixation[1] for fixation in eye_events]
        total_fixation_duration = sum(fixation_durations)
        average_fixation_duration = np.mean(fixation_durations)

        # Write the file name, number of saccades, number of fixations, total fixation duration, and average fixation duration to the CSV
        csv_writer.writerow([file_name, None, len(eye_events), total_fixation_duration, average_fixation_duration])

print(f"Results saved to {output_csv}")

Results saved to bachelorthesis/emip_fixations.csv
